In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch import Tensor
from phenom_p import IMRPhenomPv2
from phenom_d import IMRPhenomD
params={
    'text.usetex':True,
    'font.family':'serif',
    'xtick.minor.visible':True,
    'ytick.minor.visible':True,
    'xtick.top':True,
    'ytick.left':True,
    'ytick.right':True,
    'xtick.direction':'out',
    'ytick.direction':'out',
    'xtick.minor.size':2.5,
    'xtick.major.size':5,
    'ytick.minor.size':2.5,
    'ytick.major.size':5,
    'axes.axisbelow':True,
    "xtick.labelsize": 16,
    "ytick.labelsize": 16,
    "axes.labelsize": 18,
    "legend.fontsize": 14,
    "legend.title_fontsize": 16,
    "figure.titlesize": 22,
    "axes.labelpad": 10.0,
}
plt.rcParams.update(params)

In [7]:
phenom_p = IMRPhenomPv2().cuda()
phenom_d = IMRPhenomD().cuda()
chi1z = 0.5
chi2z = -0.1
mass_ratio = 1.5
chirp_mass = 30
distance = 100
tc = 0
phic = 0
inclination = 0
torch_freqs = torch.arange(
        10, 300, 1/128.
    ).cuda()
_params = torch.tensor(
        [
            chirp_mass,
            mass_ratio,
            0,
            0,
            chi1z,
            0,
            0,
            chi2z,
            distance,
            tc,
            phic,
            inclination,
        ]
    ).cuda()
f_ref = 20.

def batch_params(params, batchsize = 10):
    params = params.repeat(batchsize, 1)
    batched_chirp_mass = params[:, 0]
    batched_mass_ratio = params[:, 1]
    batched_chi1x = params[:, 2]
    batched_chi1y = params[:, 3]
    batched_chi1z = params[:, 4]
    batched_chi2x = params[:, 5]
    batched_chi2y = params[:, 6]
    batched_chi2z = params[:, 7]
    batched_distance = params[:, 8]
    batched_tc = params[:, 9]
    batched_phic = params[:, 10]
    batched_inclination = params[:, 11]
    return batched_chirp_mass, batched_mass_ratio, batched_chi1x, batched_chi1y, batched_chi1z, batched_chi2x, batched_chi2y, batched_chi2z, batched_distance, batched_tc, batched_phic, batched_inclination

In [10]:
print("PhenomP: batch size  = 100")
params = batch_params(_params, 100)
%timeit -n 10 -r 10 phenom_p.forward(torch_freqs, *params, f_ref)

PhenomP: batch size  = 100


83.8 ms ± 1.03 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [12]:
print("PhenomP: batch size  = 1000")
params = batch_params(_params, 1000)
%timeit -n 10 -r 10 phenom_p.forward(torch_freqs, *params, f_ref)

545 ms ± 1.09 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [16]:
params = batch_params(_params, 100)
params = [*params[:2], params[4], params[7], params[8], *params[10:]]
print("PhenomD: batch size  = 100")
%timeit -n 10 -r 10 phenom_d.forward(torch_freqs, *params, f_ref)

PhenomD: batch size  = 100


77 ms ± 338 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [17]:
params = batch_params(_params, 1000)
params = [*params[:2], params[4], params[7], params[8], *params[10:]]
print("PhenomD: batch size  = 1000")
%timeit -n 10 -r 10 phenom_d.forward(torch_freqs, *params, f_ref)

PhenomD: batch size  = 1000
557 ms ± 78.6 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)
